In [7]:
// Find all employees whose salary is more than average salary of department [Amazon]

val empDF = Seq(
    (101, "Ram", 10000, 105),
    (102, "Mohan", 75000, 104),
    (103, "Ramesh", 35000, 104),
    (104, "Manish", 50000, 106),
    (105, "Rakesh", 85000, 106),
    (106, "Dan", 100000, null.asInstanceOf[Int])
).toDF("emp_id","name","salary","manager_id")

val deptDF = Seq(
    (101,3),
    (102,2),
    (103,2),
    (104,2),
    (105,3),
    (106,1)
).toDF("emp_id","dept_id")

empDF.show(false)
deptDF.show(false)


empDF.createOrReplaceTempView("emp")
deptDF.createOrReplaceTempView("dept")


+------+------+------+----------+
|emp_id|name  |salary|manager_id|
+------+------+------+----------+
|101   |Ram   |10000 |105       |
|102   |Mohan |75000 |104       |
|103   |Ramesh|35000 |104       |
|104   |Manish|50000 |106       |
|105   |Rakesh|85000 |106       |
|106   |Dan   |100000|0         |
+------+------+------+----------+

+------+-------+
|emp_id|dept_id|
+------+-------+
|101   |3      |
|102   |2      |
|103   |2      |
|104   |2      |
|105   |3      |
|106   |1      |
+------+-------+



empDF: org.apache.spark.sql.DataFrame = [emp_id: int, name: string ... 2 more fields]
deptDF: org.apache.spark.sql.DataFrame = [emp_id: int, dept_id: int]


In [30]:
spark.sql("""
    SELECT e.emp_id,e.name,e.salary,d.dept_id FROM emp e INNER JOIN dept d ON e.emp_id=d.emp_id
""").createOrReplaceTempView("joined")


val avgDF = spark.sql("""
SELECT * FROM (
    SELECT 
        *,
        total_salary/total_emp avg_salary_per_dept
        FROM (
            SELECT 
                *,
                COUNT(emp_id) over(partition by dept_id) total_emp,
                SUM(salary) over(partition by dept_id) total_salary
            FROM joined
            ) t1
    )t2
    WHERE salary > avg_salary_per_dept
""")

avgDF.show(false)

+------+------+------+-------+---------+------------+-------------------+
|emp_id|name  |salary|dept_id|total_emp|total_salary|avg_salary_per_dept|
+------+------+------+-------+---------+------------+-------------------+
|102   |Mohan |75000 |2      |3        |160000      |53333.333333333336 |
|105   |Rakesh|85000 |3      |2        |95000       |47500.0            |
+------+------+------+-------+---------+------------+-------------------+



avgDF: org.apache.spark.sql.DataFrame = [emp_id: int, name: string ... 5 more fields]
